# Further Analyses- Producing 95% CIs and Examining Most Important Features

In [1]:
from IPython.core.interactiveshell import InteractiveShell
from matplotlib import pyplot 
from numpy import mean
from numpy import std
from sklearn.calibration import calibration_curve
from sklearn.metrics import confusion_matrix as confusion
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score as recall
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import resample
import joblib
import numpy as np
import pandas as pd
import warnings
InteractiveShell.ast_node_interactivity = "all"

Bootstrapped CIs for First Round of Evals (Full Test Dataset)

In [7]:
#Loading datasets
Xi_hold=pd.read_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\Xi_hold.csv')
y_hold_si=pd.read_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\processed_data\y_hold_si.csv')
y_hold_nssi=pd.read_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\processed_data\y_hold_nssi.csv')
y_hold_att=pd.read_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\processed_data\y_hold_att.csv')
y_hold_sitbs=pd.read_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\processed_data\y_hold_sitbs.csv')

In [8]:
Xi_hold.shape

(729, 1011)

In [21]:
#Defining a function to prepare 100 bootstrapped test sets, and obtain estimates of performance metrics for each resampled version of test data
algos=['LR', 'RF', 'XGB']
threshs1=[0.188868, 0.175579, 0.119263]
models1= {a: joblib.load(f'{a}_si.sav') for a in algos}

def boot(model, fulltest, thresh):
    #Suppressing warnings for this section. Warning appears because the bootstrapped samples do not have feature names as they are arrays while the model has previously been fitted on pandas dataframes with feature names
    #This is ok because the features are in the same order as the original test set
    warnings.simplefilter(action='ignore', category=UserWarning)
    n_iterations=100
    aucs=list()
    f1s=list()
    sens=list()
    specs=list()
    ppvs=list()
    values=fulltest.values
    for i in range(n_iterations):
        test=resample(values, n_samples=len(fulltest), stratify=values[:, -1], random_state=i)
        probs=model.predict_proba(test[:, :-1])
        probs=probs[:, 1]
        auc=roc_auc_score(test[:, -1], probs)
        aucs.append(auc)
        pred=np.where(probs>= thresh, 1, 0)
        f1=f1_score(test[:, -1], pred)
        f1s.append(f1)
        sen=recall(test[:, -1], pred)
        sens.append(sen)
        ppv=precision_score(test[:, -1], pred)
        ppvs.append(ppv)
        tn, fp, fn, tp=confusion(test[:, -1], pred).ravel()
        spec=tn/(tn+fp)
        specs.append(spec)
        
    metrics=["aucs", "f1s", "sens", "specs", "ppvs"]
    metricsdf=pd.DataFrame(zip(aucs, f1s, sens, specs, ppvs), columns=metrics)
    return metricsdf

#Iterating through tuples to evaluate LR, RF, and XGB Models 
#Value of the tuple algo is paired to the other corresponding value in threshs, i.e. 'LR' is paired with threshold of 0.188868
def runevals(models1, fulltest, threshs):
    for a, t in zip(algos, threshs):
        clf=models1[a]
        print(f'95% CIs for performance using {a} with threshold at {t}')
        metricsdf=boot(clf, fulltest, t)
        print(metricsdf.quantile([.025, .975]).round(3))
        print('\n')

In [10]:
fulltest_si=Xi_hold.join(y_hold_si)
runevals(models1, fulltest_si, threshs1)

95% CIs for performance using LR with threshold at 0.188868
           aucs       f1s    sens     specs      ppvs
0.025  0.694001  0.308598  0.3750  0.816746  0.262009
0.975  0.786711  0.447163  0.5625  0.864968  0.367685


95% CIs for performance using RF with threshold at 0.175579
           aucs       f1s      sens     specs      ppvs
0.025  0.777556  0.407189  0.624479  0.754305  0.295607
0.975  0.865730  0.509746  0.781250  0.818404  0.385700


95% CIs for performance using XGB with threshold at 0.119263
           aucs       f1s      sens     specs      ppvs
0.025  0.783426  0.385365  0.760417  0.654779  0.256433
0.975  0.866916  0.467795  0.901302  0.723618  0.320392




In [11]:
threshs2=[0.138621, 0.160344, 0.172887]
models2= {a: joblib.load(f'{a}_nssi.sav') for a in algos}

fulltest_nssi=Xi_hold.join(y_hold_nssi)
runevals(models2, fulltest_nssi, threshs2)

95% CIs for performance using LR with threshold at 0.138621
           aucs       f1s      sens     specs      ppvs
0.025  0.655818  0.224323  0.321429  0.855869  0.170443
0.975  0.808005  0.385919  0.553571  0.900446  0.293034


95% CIs for performance using RF with threshold at 0.160344
           aucs       f1s      sens     specs      ppvs
0.025  0.771589  0.315188  0.544196  0.821620  0.217391
0.975  0.888752  0.445714  0.795089  0.870022  0.313881


95% CIs for performance using XGB with threshold at 0.172887
           aucs       f1s      sens     specs      ppvs
0.025  0.769216  0.302383  0.391964  0.895988  0.248378
0.975  0.879479  0.511278  0.670089  0.936107  0.437224




In [12]:
threshs3=[0.074037, 0.073374, 0.087420]
models3= {a: joblib.load(f'{a}_att.sav') for a in algos}

fulltest_att=Xi_hold.join(y_hold_att)
runevals(models3, fulltest_att, threshs3)

95% CIs for performance using LR with threshold at 0.074037
           aucs       f1s      sens     specs      ppvs
0.025  0.690694  0.126141  0.296094  0.819907  0.080938
0.975  0.846409  0.267160  0.625000  0.869440  0.171988


95% CIs for performance using RF with threshold at 0.073374
           aucs       f1s      sens     specs     ppvs
0.025  0.800614  0.174382  0.562500  0.764562  0.10436
0.975  0.915505  0.280363  0.860156  0.817109  0.16923


95% CIs for performance using XGB with threshold at 0.08742
           aucs       f1s     sens     specs      ppvs
0.025  0.795508  0.224842  0.43750  0.873745  0.151437
0.975  0.910338  0.381892  0.78125  0.909003  0.254234




In [13]:
threshs4=[0.208806, 0.152130, 0.176893]
models4= {a: joblib.load(f'{a}_sitbs.sav') for a in algos}

fulltest_sitbs=Xi_hold.join(y_hold_sitbs)
runevals(models4, fulltest_sitbs, threshs4)

95% CIs for performance using LR with threshold at 0.208806
           aucs       f1s      sens     specs      ppvs
0.025  0.666220  0.379787  0.504132  0.750699  0.296542
0.975  0.756427  0.477128  0.624174  0.821752  0.392540


95% CIs for performance using RF with threshold at 0.15213
           aucs       f1s      sens     specs      ppvs
0.025  0.772562  0.431631  0.702479  0.672615  0.309099
0.975  0.845087  0.521524  0.830785  0.735197  0.380550


95% CIs for performance using XGB with threshold at 0.176893
           aucs       f1s      sens     specs      ppvs
0.025  0.781079  0.452331  0.632025  0.750781  0.346575
0.975  0.851117  0.545517  0.772934  0.810855  0.429257




Bootstrapped CIs for Second Round Evals (Reduced Test Dataset)

In [14]:
#Loading the reduced datasets created in the evaluation notebook
si_test=pd.read_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\processed_data\si_test.csv')
nssi_test=pd.read_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\processed_data\nssi_test.csv')
att_test=pd.read_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\processed_data\att_test.csv')

In [15]:
si_test=si_test.drop(columns=['nssi', 'att', 'sitbs'])
nssi_test=nssi_test.drop(columns=['si', 'att', 'sitbs'])
att_test=att_test.drop(columns=['si', 'nssi', 'sitbs'])

In [22]:
#Evaluating models for predicting SI
runevals(models1, si_test, threshs1)
runevals(models4, si_test, threshs1)

95% CIs for performance using LR with threshold at 0.188868
        aucs    f1s   sens  specs   ppvs
0.025  0.693  0.328  0.395  0.816  0.266
0.975  0.791  0.456  0.573  0.873  0.396


95% CIs for performance using RF with threshold at 0.175579
        aucs    f1s   sens  specs   ppvs
0.025  0.792  0.422  0.624  0.757  0.313
0.975  0.872  0.527  0.792  0.822  0.398


95% CIs for performance using XGB with threshold at 0.119263
        aucs    f1s   sens  specs   ppvs
0.025  0.796  0.403  0.750  0.659  0.272
0.975  0.870  0.484  0.901  0.734  0.336


95% CIs for performance using LR with threshold at 0.188868
        aucs    f1s   sens  specs   ppvs
0.025  0.685  0.353  0.557  0.714  0.255
0.975  0.791  0.444  0.729  0.785  0.328


95% CIs for performance using RF with threshold at 0.175579
        aucs    f1s   sens  specs   ppvs
0.025  0.798  0.403  0.692  0.701  0.278
0.975  0.871  0.489  0.854  0.765  0.345


95% CIs for performance using XGB with threshold at 0.119263
        aucs 

In [23]:
#Predicting NSSI
runevals(models2, nssi_test, threshs2)
runevals(models4, nssi_test, threshs2)

95% CIs for performance using LR with threshold at 0.138621
        aucs    f1s   sens  specs   ppvs
0.025  0.680  0.278  0.339  0.877  0.229
0.975  0.825  0.437  0.563  0.922  0.360


95% CIs for performance using RF with threshold at 0.160344
        aucs    f1s   sens  specs   ppvs
0.025  0.796  0.371  0.544  0.856  0.273
0.975  0.909  0.526  0.795  0.905  0.411


95% CIs for performance using XGB with threshold at 0.172887
        aucs    f1s   sens  specs   ppvs
0.025  0.794  0.383  0.411  0.914  0.352
0.975  0.905  0.585  0.661  0.949  0.537


95% CIs for performance using LR with threshold at 0.138621
        aucs    f1s   sens  specs   ppvs
0.025  0.704  0.228  0.687  0.577  0.136
0.975  0.839  0.290  0.875  0.655  0.176


95% CIs for performance using RF with threshold at 0.160344
        aucs    f1s   sens  specs   ppvs
0.025  0.799  0.287  0.723  0.682  0.177
0.975  0.905  0.375  0.911  0.739  0.239


95% CIs for performance using XGB with threshold at 0.172887
        aucs 

In [24]:
#Predicting attempts
runevals(models3, att_test, threshs3)
runevals(models4, att_test, threshs3)

95% CIs for performance using LR with threshold at 0.074037


        aucs    f1s   sens  specs   ppvs
0.025  0.701  0.143  0.281  0.832  0.094
0.975  0.846  0.305  0.656  0.881  0.203


95% CIs for performance using RF with threshold at 0.073374
        aucs    f1s   sens  specs   ppvs
0.025  0.827  0.232  0.562  0.805  0.145
0.975  0.932  0.356  0.875  0.864  0.228


95% CIs for performance using XGB with threshold at 0.08742
        aucs    f1s   sens  specs   ppvs
0.025  0.827  0.286  0.438  0.896  0.210
0.975  0.938  0.525  0.829  0.942  0.406


95% CIs for performance using LR with threshold at 0.074037
        aucs    f1s   sens  specs   ppvs
0.025  0.657  0.113  0.812  0.290  0.060
0.975  0.827  0.132  0.985  0.356  0.071


95% CIs for performance using RF with threshold at 0.073374
        aucs    f1s  sens  specs   ppvs
0.025  0.825  0.151   1.0  0.410  0.082
0.975  0.929  0.171   1.0  0.489  0.093


95% CIs for performance using XGB with threshold at 0.08742
        aucs    f1s   sens  specs   ppvs
0.025  0.789  0.163  0.765  0.568  0.